In [1]:
from trainingData import  TrainingData
from preprocessing2 import  datasetMinMaxTraining
from preprocessing2 import  datasetMinMaxTesting
from preprocessing2 import  dataset
from preprocessing2 import  datasetTesting as dfTesting
from preprocessing2 import  datasetTraining as dfTraining
from backProp import backProp
from predict import Predict
from util import Function
from trainingNetwork import TrainingNetwork
from math import exp
import math
import random
import numpy as np
import pdb

/Users/chrstevan/Documents/SKRIPSI 2/Skripsi2-1/preprocessing2.py:52: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  datasetMinMaxTraining['kategoriScale'][i]=kategoriScale
/Users/chrstevan/Documents/SKRIPSI 2/Skripsi2-1/preprocessing2.py:52:

In [2]:
# scaleWeightInputToHidden = []
# scaleWeightHiddenToOutput =[]
# totalError =[]

In [3]:
def trainingNetwork(reqCol,reqVar,reqHiddenLayer,reqEpoch,lrate,reqTargetError,reqInput,reqColScale):
    scaleWeightInputToHidden = []
    scaleWeightHiddenToOutput =[]
    totalError =[]
    
    
    count_epoch = 0
    StatusError = 0
    n_fold = 0
    checkerror = 0
    
    weightInputToHidden = Function.inputWeight(reqHiddenLayer,reqVar)
    weightHiddentoOutput =Function.inputWeight(reqHiddenLayer,0)   
   
    # weightInputToHidden = np.array(weightInputToHidden,dtype='float64')
    # weightHiddentoOutput = np.array(weightHiddentoOutput, dtype='float64')

    while count_epoch < reqEpoch and StatusError !=1:
        for i in range(len(datasetMinMaxTraining)):
            # Collect Data from Dataset after normalization
            tempInputSCale,colScale= Function.chooseColumn(reqCol,datasetMinMaxTraining,i,reqColScale)
            breakpoint()
            
            oTransferACtivation,tmpHiddenValue,tmpCount = TrainingNetwork.forwardPropagation(reqWeightInputToHidden=weightInputToHidden,reqWeightHiddentoOutput=weightHiddentoOutput,reqInput=tempInputSCale)
            
            errorValue = ((colScale-oTransferACtivation)**2)
            delta0 = (colScale - oTransferACtivation) * oTransferACtivation * (1 - oTransferACtivation)
            breakpoint()
            
            ## BackPropagation
            #     breakpoint()
            newWeightInputToHidden,newWeightHiddenToOutput=TrainingNetwork.backPropagation(reqTmpHiddenValue=tmpHiddenValue,reqHiddenLayer=reqHiddenLayer,reqWeightHiddentoOutput=weightHiddentoOutput,reqWeightInputToHidden=weightInputToHidden,reqTempInputScale=tempInputSCale,reqLrate=lrate,reqDelta0=delta0,reqJ=tmpCount)


            weightInputToHidden = newWeightInputToHidden
            weightHiddentoOutput = newWeightHiddenToOutput
            breakpoint()
        
        if errorValue > checkerror:
            n_fold += 1
            
        if errorValue < reqTargetError and count_epoch > 100 or n_fold > 100 or count_epoch >= reqEpoch:
            currentError = errorValue
            scaleWeightInputToHidden=weightInputToHidden
            scaleWeightHiddenToOutput=weightHiddentoOutput
            StatusError = 1
            totalError.append(currentError)     
        
        checkerror = errorValue
        
        print("Epoch : ",count_epoch,"Errors : ",errorValue,' Status Error:',StatusError,' Fold:',n_fold)

        count_epoch += 1
    return scaleWeightHiddenToOutput,scaleWeightInputToHidden,totalError 

In [4]:
def trainingData(reqCol,reqDataset,reqScaleWeightHiddenToOutput,reqScaleWeightInputToHidden,reqDatasetReal,reqColPredict):
    accCount = 0
    predictedValue =[]
    maxColPredict =max(reqDatasetReal[reqColPredict])
    minColPredict =min(reqDatasetReal[reqColPredict])
    for i in range(len(reqDataset)):
        # Collect Data from Dataset after normalization
        tempInputScale= Function.chooseColumnTrainingData(reqCol,reqDataset,i)
        tempArrHidden =[]  
        tempArrHidden.append(1.0)  
        for j in range(len(reqScaleWeightInputToHidden)):
            tempArrWeight = reqScaleWeightInputToHidden[j]
            activation = 0.0
            transfer_activation = 0.0
            for k in range(len(tempArrWeight)):
                tempWeight = tempArrWeight[k]
                tempInput = tempInputScale[k]
                tempActivation = tempWeight * tempInput
                activation = activation + tempActivation
            transfer_activation = 1.0 / (1.0 + pow(exp(1),(activation * -1 )))
            tempArrHidden.append(transfer_activation)
        output_net = 0.0
        for l in range(len(reqScaleWeightHiddenToOutput)):
            tempWeightHidden = reqScaleWeightHiddenToOutput[l]
            hidden_value = tempArrHidden[l]
            temp_output_net = tempWeightHidden * hidden_value
            output_net = output_net + temp_output_net
        output = 1.0 / (1.0 + pow(exp(1),(output_net * -1 )))
        realOutput = (output * (maxColPredict - minColPredict))+minColPredict
        predictedValue.append(realOutput)
    return predictedValue

In [5]:
reqColumn = ['negoScale','jualScale', 'beliScale','qtyScale','suhuScale','kategoriScale']
targetError = 0.001
lrate =0.5
epoch =3000
inputValue = 6911

In [6]:
# scaleWeightHiddenToOutput,scaleWeightInputToHidden= trainingNetwork(6,7,10,0.5,0.05)
targetError = 0.001
lrate =0.5
scaleWeightHiddenToOutputBeli,scaleWeightInputToHiddenBeli,totalErrorBeli=backProp.trainingNetwork(reqCol= reqColumn,reqInput=inputValue,reqEpoch=epoch,reqHiddenLayer=len(reqColumn)+1,reqTargetError=targetError,reqVar=len(reqColumn),lrate=lrate,reqColScale='beliScale')

Epoch :  0 Errors :  0.000728625033748504  Status Error: 1  Fold: 1


In [7]:
targetError = 0.006
lrate =0.8
scaleWeightHiddenToOutputJual,scaleWeightInputToHiddenJual,totalErrorJual=backProp.trainingNetwork(reqCol= reqColumn,reqInput=inputValue,reqEpoch=epoch,reqHiddenLayer=len(reqColumn)+1,reqTargetError=targetError,reqVar=len(reqColumn),lrate=lrate,reqColScale='jualScale')

Epoch :  0 Errors :  0.004246997057666482  Status Error: 1  Fold: 1


In [8]:
targetError = 0.0061
lrate =0.8
scaleWeightHiddenToOutputNego,scaleWeightInputToHiddenNego,totalErrorNego=backProp.trainingNetwork(reqCol= reqColumn,reqInput=inputValue,reqEpoch=epoch,reqHiddenLayer=len(reqColumn)+1,reqTargetError=targetError,reqVar=len(reqColumn),lrate=lrate,reqColScale='negoScale')

Epoch :  0 Errors :  0.004143404695042421  Status Error: 1  Fold: 1


In [9]:
totalErrorNego

[np.float64(0.004143404695042421)]

In [10]:
scaleWeightInputToHiddenNego

[[np.float64(0.003967749453680998),
  np.float64(0.24433164190971002),
  np.float64(0.19681482966749134),
  np.float64(0.19836665030328993),
  np.float64(1.0223944110891225),
  np.float64(0.9155881227115192),
  np.float64(0.8272883789052049)],
 [np.float64(0.003966288033865849),
  np.float64(0.24424164855021915),
  np.float64(0.1967423379198775),
  np.float64(0.198293586981928),
  np.float64(1.022017838054785),
  np.float64(0.9152508890629066),
  np.float64(0.8269836682262952)],
 [np.float64(0.003967117403185532),
  np.float64(0.24429272061764512),
  np.float64(0.1967834776599973),
  np.float64(0.19833505109545896),
  np.float64(1.0222315467496188),
  np.float64(0.9154422722909324),
  np.float64(0.8271565943669208)],
 [np.float64(0.0039675107023129375),
  np.float64(0.2443169397430416),
  np.float64(0.19680298672972868),
  np.float64(0.19835471398783192),
  np.float64(1.02233289055533),
  np.float64(0.9155330290319839),
  np.float64(0.8272385984857126)],
 [np.float64(0.0039669457026375

In [11]:
reqColToPredictJual ='jual'
predictedjualScale =trainingData(reqCol=reqColumn,reqDataset=datasetMinMaxTraining,reqScaleWeightHiddenToOutput=scaleWeightHiddenToOutputJual,reqScaleWeightInputToHidden=scaleWeightInputToHiddenJual,reqDatasetReal=dfTraining,reqColPredict=reqColToPredictJual)

In [12]:
predictedjualScale

[np.float64(1219454.949452677),
 np.float64(1238948.209800791),
 np.float64(1254799.1599799905),
 np.float64(1254632.2841741906),
 np.float64(1326526.9338728408),
 np.float64(1251937.2500586694),
 np.float64(1182714.8306318019),
 np.float64(1234786.2534755142),
 np.float64(1233259.3339216944),
 np.float64(1252897.189809793),
 np.float64(1249509.92209477),
 np.float64(1297721.9433056987),
 np.float64(1305350.6041713506),
 np.float64(1288279.475262006),
 np.float64(1277447.5125994666),
 np.float64(1250949.8634218639),
 np.float64(1224810.0152887176),
 np.float64(1249075.0618508484),
 np.float64(1221231.3336233564),
 np.float64(1252684.8938358554),
 np.float64(1276161.0866116874),
 np.float64(1282655.9055496326),
 np.float64(1234156.5715960043),
 np.float64(1258816.7025583987),
 np.float64(1259801.1114407985),
 np.float64(1257247.7423160914),
 np.float64(1207635.6148315219),
 np.float64(1222458.354478404),
 np.float64(1223256.417418299),
 np.float64(1205971.336324236),
 np.float64(1198524

In [13]:
reqColToPredictBeli ='beli'
predictedBeliScale =trainingData(reqCol=reqColumn,reqDataset=datasetMinMaxTraining,reqScaleWeightHiddenToOutput=scaleWeightHiddenToOutputBeli,reqScaleWeightInputToHidden=scaleWeightInputToHiddenBeli,reqDatasetReal=dfTraining,reqColPredict=reqColToPredictBeli)

In [14]:
reqColToPredictNego ='nego'
predictedNegoScale =trainingData(reqCol=reqColumn,reqDataset=datasetMinMaxTraining,reqScaleWeightHiddenToOutput=scaleWeightHiddenToOutputNego,reqScaleWeightInputToHidden=scaleWeightInputToHiddenNego,reqDatasetReal=dfTraining,reqColPredict=reqColToPredictNego)

In [15]:
accuracyValue,datasetShow = Predict.predictAccuracy(reqDatasetReal=dfTesting,reqChooseAccuracyColumnNego=reqColToPredictNego,reqChooseAccuracyColumnJual=reqColToPredictJual,reqChooseAccuracyColumnBeli=reqColToPredictBeli,reqPredictValueNego=predictedNegoScale,reqPredictValueJual=predictedjualScale,reqPredictValueBeli=predictedBeliScale)

In [16]:
accuracyValue

np.float64(-1939.4427424470477)

In [17]:
datasetShow

,Produk,Real Price Buy,Predicted Price Buy,Real Price Sell,Predicted Price Sell,Real Price Nego,Predicted Price Nego
0,BASO-30 BIRU,22500,1021806.15,25000,1256671.62,30000,1047998.41
1,CENTRAL DELUXE PT 160,1700000,947998.84,1900000,1219454.95,1860000,1000113.64
2,Meja napolly mapan 45 jati,95000,932373.54,95000,1238948.21,110000,993860.06
3,TCC 500 JATI,86700,1028727.65,90000,1254799.16,100000,1091597.25
4,Boxer B5-4,25000,1028661.29,25000,1254632.28,40000,1091389.89
...,...,...,...,...,...,...,...
1723,M.Com Miyako 606SBL,186000,1081723.16,240000,1265989.11,220000,1113041.74
1724,Bantal Silikon Motif,26000,1098963.42,37500,1249354.05,50000,1120032.84
1725,Guling Silikon Motif,27000,1092883.36,35000,1232314.53,50000,1106164.22
1726,Elite Serenity Eternity Deluxe PT 160 x 200,1640000,1088766.08,2300000,1216810.93,2150000,1096097.19


In [18]:
tempErrorBeli = math.sqrt(totalErrorBeli[0])
tempErrorTotalBeli = '%.*f' % (3, tempErrorBeli)
errorTotalBeli =(float(tempErrorTotalBeli) * 100)
tempErrorJual = math.sqrt(totalErrorJual[0])
tempErrorTotalJual = '%.*f' % (3, tempErrorJual)
errorTotalJual =(float(tempErrorTotalJual) * 100)
tempErrorNego = math.sqrt(totalErrorNego[0])
tempErrorTotalNego = '%.*f' % (3, tempErrorNego)
errorTotalNego =(float(tempErrorTotalNego) * 100)

temptotalErrorAll = (tempErrorBeli+tempErrorJual+tempErrorNego)/3
tempPredict = '%.*f' % (3, temptotalErrorAll)

valueError = 100-(float(tempPredict)*100)

In [19]:
temptotalErrorAll*100

5.217710917955283

In [20]:
valueError

94.8